In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sstats
import scipy as sp

In [3]:
Q_cunt = np.array([
    [-0.0085,0.005,0.0025,0,0.001],
    [0,-0.014,0.005,0.004,0.005],
    [0,0,-0.008,0.003,0.005],
    [0,0,0,-0.009,0.009],
    [0,0,0,0,0]])

x_start = 0
nrSamples = 1000


def continuousMarkovSampling(nrSamples,Q,X_end = 4,x_start = 0):
    CMC_samples = []

    for i in range(nrSamples):
        T = 0
        Xi = x_start
        sample = []

        while(Xi != X_end):
            P_current = Q[Xi]
            t_sojourn = np.random.exponential(-1*(1/P_current[Xi]))
            sample += [(Xi,T,t_sojourn)]
            T += t_sojourn
            #Pick the next state
            Xi = np.random.choice([i for i in range(Xi+1,5)],p=-1*P_current[Xi+1:]/P_current[Xi])


        #Add the last sample - makes data processing easier later
        P_current = Q[Xi]
        t_sojourn = np.exp(P_current[Xi])
        sample += [(Xi,T,t_sojourn)]

        CMC_samples += [sample]

    return CMC_samples

samples = continuousMarkovSampling(nrSamples,Q_cunt,4,0)

In [4]:
def getX48(Q,X_48,x_start):
    T = 0
    Xi = x_start
    sample = []

    while(T<48 and not(Xi == 4)):
        P_current = Q[Xi]
        t_sojourn = np.random.exponential(-1*(1/P_current[Xi]))
        sample += [(Xi,T,t_sojourn)]
        T += t_sojourn
        #Pick the next state
        Xi = np.random.choice([i for i in range(Xi+1,5)],p=-1*P_current[Xi+1:]/P_current[Xi])

    if(T<48):
        sample += [(Xi,T,0)]

    if(sample[-1][0] == X_48):
        return sample
    else:
        return []

In [5]:
def getCheckupTS(woman_life):
    Y = []
    i = 0
    max_Ti = woman_life[-1][1]
    alive = True
    while(alive):
        yi = i*48

        if(yi >= max_Ti):
            alive = False
            Y += [woman_life[-1][0]]
        else:
            Y += [Xi for (Xi,Ti,ti) in woman_life if ((Ti<=yi) and ((Ti+ti)>=yi))]

        i += 1

    return Y


In [6]:
checkups = [getCheckupTS(X) for X in samples]

Based off of our histograms we get the feeling, that the overall process follows an exponential distribution.

We therefore make the assumption, that the sojourn time is also exponentially distributed.

In [28]:
# Initialize the Q matrix with zeros
Q_est = np.zeros((5, 5))

for i in range(5):
    for j in range(i+1, 5):
        #Q_est[i, j] = np.random.exponential(0.0001)
        Q_est[i, j] = np.random.exponential(0.01)
# Fill in the diagonal elements
for i in range(4):
    Q_est[i, i] = -np.sum(Q_est[i])

# The last row remains as zeros
Q_est[4, 4] = 0

# Display the resulting matrix
Q_est


array([[-0.03533661,  0.01914702,  0.00107135,  0.01488267,  0.00023557],
       [ 0.        , -0.02144029,  0.00109367,  0.01250988,  0.00783673],
       [ 0.        ,  0.        , -0.01404694,  0.01145887,  0.00258807],
       [ 0.        ,  0.        ,  0.        , -0.03264828,  0.03264828],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [39]:
error = 1000000

def getSimulatedTrajectories(checkups,Qk):

    trajectories = []
    for wi,w in enumerate(checkups):

        traj_i = []
        for i,Xi in enumerate(w):
            sameEndResult = False
            #print(f"Attempting: {w[i-1]} -> {Xi}")
            if(i != 0):
                sim_result = []
                count_wrongs = 0
                while(len(sim_result) == 0):
                    count_wrongs += 1
                    sim_result = getX48(Qk,Xi,w[i-1])
                #print(f"Took us this many attempts: {count_wrongs} to go from {w[i-1]} -> {Xi}")
                traj_i += sim_result

        trajectories += [traj_i]

    return trajectories


def updateQk(trajectories):
    Nij = np.zeros((5,5))
    Si = np.zeros(5)
    Q_k_1 = np.zeros((5,5))

    for w in trajectories:
        for idx,(Xi,Ti,ti) in enumerate(w):
            if(Xi != 4):
                X2 = w[idx + 1][0]
                Nij[Xi][X2] += 1
                Si[Xi] += np.minimum(48.0,ti)

    for i in range(4):
        Q_k_1[i] = Nij[i]/Si[i]

    #Normalize
    for i in range(4):
        Q_k_1[i, i] = -np.sum(Q_k_1[i]) + Q_k_1[i,i]

    return Q_k_1


Qs = []
err = []
latest_err = 100
Qk = Q_est
while(latest_err > 10**(-3)):
    trajectories = getSimulatedTrajectories(checkups,Qk)
    Q_new = updateQk(trajectories)
    Qs += [Q_new]
    latest_err = np.sum(np.abs(Qk - Q_new))
    err += [latest_err]
    Qk = Q_new

print(err)




[0.13110176960853157, 0.01192492601856214, 0.007662728822441956, 0.005690760074123705, 0.002931439622462938, 0.0029316451965397056, 0.0012778729812726204, 0.0018641847980552816, 0.0015861348947157077, 0.0013766916721863003, 0.001965276217991147, 0.002077762036810455, 0.002602312214316526, 0.0009111667877723003]


In [55]:
Q_best = Qs[-1]
Q_abs_diff = [np.abs(-Q_best[i]/Q_best[i][i] + Q_cunt[i]/Q_cunt[i][i]) for i in range(4)]

print(f"Summed abs error: {100*np.sum(Q_abs_diff)}% and max error: {np.max(Q_abs_diff)*100}% and mean error {100*np.mean(Q_abs_diff)}%")


Summed abs error: 18.800944098118052% and max error: 3.863751906456531% and mean error 0.9400472049059025%


#Task 13

Thoughts:

We see that we converge to the real Markov Chain very quickly, in fact our first guess has an absolute summed error of approx. 0.02 with a error of 0.005